In [10]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

#### 默认情况下，工具通过检查函数签名来推断参数模式。对于更严格的需求，可以指定自定义输入模式，以及自定义验证逻辑。
#### 在Python函数中，函数签名（function signature）指的是函数的声明部分，包括函数名、参数列表和返回值类型。函数签名描述了函数的输入和输出，帮助开发者理解函数的用途和使用方式。

In [11]:
from typing import Any, Dict
from langchain.agents import AgentType, initialize_agent
from langchain.llms import OpenAI
from langchain.tools.requests.tool import RequestsGetTool,TextRequestsWrapper
from pydantic import BaseModel, Field, root_validator

In [12]:
llm = OpenAI(temperature=0)

In [13]:
import tldextract

_APPROVED_DOMAINS = {
    "langchain",
    "wikipedia",
}


class ToolInputSchema(BaseModel):
    url: str = Field(...)

    @root_validator
    def validate_query(cls, values: Dict[str, Any]) -> Dict:
        url = values["url"]
        domain = tldextract.extract(url).domain
        if domain not in _APPROVED_DOMAINS:
            raise ValueError(
                f"Domain {domain} is not on the approved list:"
                f" {sorted(_APPROVED_DOMAINS)}"
            )
        return values


tool = RequestsGetTool(
    args_schema=ToolInputSchema, requests_wrapper=TextRequestsWrapper()
)

In [14]:
agent = initialize_agent(
    [tool], llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False
)

In [15]:
# This will succeed, since there aren't any arguments that will be triggered during validation
answer = agent.run("What's the main title on langchain.com?")
print(answer)

The main title on langchain.com is "LANG CHAIN 🦜️🔗 Official Home Page".


In [16]:
agent.run("What's the main title on google.com?")

ValidationError: 1 validation error for ToolInputSchema
__root__
  Domain google is not on the approved list: ['langchain', 'wikipedia'] (type=value_error)

In [17]:
answer = agent.run("What's the main title on wikipedia.com?")
print(answer)

InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 35461 tokens (35205 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.